# Assignment 3:  SQL JOINS

In [1]:
%load_ext sql
%sql postgres://isdb@localhost/postgres

'Connected: isdb@postgres'

## Baby Names

### What are the most popular gender neutral names?

This part of the assignment uses the `baby names` data set we looked at the very beginning of the course.  In the assignment folder you will find the file `babynames_load.sql` and `babynames_all.csv`. Copy them both to your assignment folder. You can load the data file with:
```
% psql -d postgres -U isdb -f babynames_load.sql
```

First, a few simple warm ups to check that the data is alright

In [2]:
%sql select * from baby_names limit 0;

 * postgres://isdb@localhost/postgres
0 rows affected.


state,gender,year,fname,number


In [3]:
%sql select count(*) from baby_names;

 * postgres://isdb@localhost/postgres
1 rows affected.


count
5933561


Check that you get 5,933,561 rows.  Note that this **not** the total number of babies born.  For that we need to `sum(count)`.

In [4]:
%%sql

SELECT sum(number)
  FROM Baby_names

 * postgres://isdb@localhost/postgres
1 rows affected.


sum
308180519


The total number of babies should be 308,180,519

While the number of babies born from 1910 to 2015 is large, how many distinct baby names have been used? 

The below SQL expression computes this. It may take a little while to compute as there are 5.7 Million rows. (Around a minute.) 

After you have computed and checked the result, you can **comment out the below query**, to avoid repeatedly calculating it. You can convert the cell to raw mode.

You should have received a count of 31314.

In a similar way you can count the number of Male names and Female names.  You will find them to be 13656 (male) and 20669 (female).  If we do the arithmetic we get 13656 + 20669 = 34325.  Which means 34325 - 31314 = 3011 names that were used for both males and females.

**Of these 3011 names, which are the most common gender neutral names?**

First create two tables to keep track of the number of male babies for each baby name.  i.e.,  how many male `Johns` were born?  how many male `Andrews` were born etc.  

Do the same for the number of female babies for each baby name.

In [5]:
%%sql

DROP TABLE IF EXISTS male_names;

CREATE TABLE male_names (fname, sum) as
    SELECT fname, sum(number) AS sum
      FROM Baby_Names
     WHERE gender = 'M'
     GROUP BY fname;       

DROP TABLE IF EXISTS female_names;

CREATE TABLE female_names (fname, sum) as
    SELECT fname, sum(number) AS sum
      FROM Baby_Names
     WHERE gender = 'F'
     GROUP BY fname;


 * postgres://isdb@localhost/postgres
Done.
13656 rows affected.
Done.
20669 rows affected.


[]

As a check, if you count the number of rows in `male_names` and `female_names` you should get 13656 (male) and 20669 (female).  [You don't need to do an explicit count; you should have seen these numbers displayed in terms of the number of rows affected with the intermediate tables were created.]

If we do an inner join on `male_names` and `female_names` then we will get a table with names and the number of times the names have been given to males and the number of times the names has been given to females.

_We will define a name to be gender neutral if the ratio of the number of times it is used as a female name and a male name lies between 0.25 and 4 i.e., f:m or m:f is 1:4 or 4:1._

Using this definition, filter the result of the join and order by the total number of babies with that name.  The first 10 entries of this table are given below

In [6]:
%%sql
-- solution
-- write your final SQL query here

DROP TABLE IF EXISTS Gender_Neutral_Names;

CREATE TABLE Gender_Neutral_Names AS
    SELECT f.fname, f.sum AS "Female Sum", m.sum AS "Male Sum", f.sum + m.sum AS "Total", 
           CAST(f.sum AS DOUBLE PRECISION)/m.sum AS "Ratio"
      FROM male_names AS m
      JOIN female_names AS f
        ON m.fname = f.fname 
     WHERE CAST (m.sum AS DOUBLE PRECISION) / f.sum > .25
       AND CAST (m.sum AS DOUBLE PRECISION) / f.sum < 4

     ORDER BY "Total" DESC;
        

SELECT * FROM Gender_Neutral_Names LIMIT 10;


 * postgres://isdb@localhost/postgres
Done.
642 rows affected.
10 rows affected.


fname,Female Sum,Male Sum,Total,Ratio
Willie,121144,412053,533197,0.294001014432609
Jordan,129236,366264,495500,0.352849310879584
Taylor,317124,105387,422511,3.00913774943779
Leslie,263363,103893,367256,2.53494460647012
Jamie,265301,82135,347436,3.2300602666342
Angel,92522,224298,316820,0.412495876022078
Lee,55612,215388,271000,0.258194514086207
Dana,188381,48697,237078,3.86843132020453
Jessie,130056,99207,229263,1.31095588012943
Marion,163362,63342,226704,2.57904707776831


In [7]:
print(_)

+--------+------------+----------+--------+-------------------+
| fname  | Female Sum | Male Sum | Total  |       Ratio       |
+--------+------------+----------+--------+-------------------+
| Willie |   121144   |  412053  | 533197 | 0.294001014432609 |
| Jordan |   129236   |  366264  | 495500 | 0.352849310879584 |
| Taylor |   317124   |  105387  | 422511 |  3.00913774943779 |
| Leslie |   263363   |  103893  | 367256 |  2.53494460647012 |
| Jamie  |   265301   |  82135   | 347436 |  3.2300602666342  |
| Angel  |   92522    |  224298  | 316820 | 0.412495876022078 |
|  Lee   |   55612    |  215388  | 271000 | 0.258194514086207 |
|  Dana  |   188381   |  48697   | 237078 |  3.86843132020453 |
| Jessie |   130056   |  99207   | 229263 |  1.31095588012943 |
| Marion |   163362   |  63342   | 226704 |  2.57904707776831 |
+--------+------------+----------+--------+-------------------+
